In [1]:
import json
import ipywidgets as widgets
from IPython.display import display
import pdfplumber

from llm_agent import chat_llm


In [2]:
with open("roles.json") as f:
    ROLE_DATA = json.load(f)

domains = list(ROLE_DATA.keys())


In [3]:
resume_upload = widgets.FileUpload(accept=".pdf", multiple=False)

typed_skills_box = widgets.Textarea(
    placeholder="Optional: Type your skills (Python, Linux, SQL...)",
    description="Your Skills:",
    layout=widgets.Layout(width="80%")
)

domain_dd = widgets.Dropdown(options=domains, description="Domain:")
role_dd = widgets.Dropdown(description="Role:")

chat_box = widgets.Textarea(
    placeholder="Talk to the AI mentor here...",
    layout=widgets.Layout(width="80%", height="80px")
)

send_btn = widgets.Button(description="Send", button_style="success")
chat_output = widgets.Output()


In [4]:
def update_roles(change=None):
    roles = list(ROLE_DATA[domain_dd.value].keys())
    role_dd.options = roles
    role_dd.value = roles[0]

domain_dd.observe(update_roles, names="value")
update_roles()


In [5]:
chat_history = []
resume_text = ""


In [6]:
def send_message(b):
    global resume_text

    user_message = chat_box.value.strip()
    if not user_message:
        return

    with chat_output:
        print("🧑 You:", user_message)

    chat_history.append({"role": "user", "content": user_message})

    # Read resume if uploaded
    if resume_upload.value:
        uploaded = resume_upload.value[0]
        with open("resume.pdf", "wb") as f:
            f.write(uploaded["content"])

        resume_text = ""
        with pdfplumber.open("resume.pdf") as pdf:
            for page in pdf.pages:
                if page.extract_text():
                    resume_text += page.extract_text()

    typed_skills = typed_skills_box.value.strip()
    required_skills = ROLE_DATA[domain_dd.value][role_dd.value]

    reply = chat_llm(
        chat_history=chat_history,
        resume_text=resume_text,
        typed_skills=typed_skills,
        domain=domain_dd.value,
        role=role_dd.value,
        required_skills=required_skills
    )

    chat_history.append({"role": "assistant", "content": reply})

    with chat_output:
        print("🤖 AI:", reply)

    chat_box.value = ""


In [7]:
send_btn.on_click(send_message)


In [8]:
display(
    resume_upload,
    typed_skills_box,
    domain_dd,
    role_dd,
    chat_box,
    send_btn,
    chat_output
)


FileUpload(value=(), accept='.pdf', description='Upload')

Textarea(value='', description='Your Skills:', layout=Layout(width='80%'), placeholder='Optional: Type your sk…

Dropdown(description='Domain:', options=('Cybersecurity', 'Software Engineering', 'Artificial Intelligence & M…

Dropdown(description='Role:', options=('SOC Analyst', 'Penetration Tester', 'Digital Forensics Analyst', 'Clou…

Textarea(value='', layout=Layout(height='80px', width='80%'), placeholder='Talk to the AI mentor here...')

Button(button_style='success', description='Send', style=ButtonStyle())

Output()